# Prediction
Sample file to test prediction results in the Web GUI

In [1]:
%matplotlib inline 

import math
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn

import model
import model_utils
import preprocessing_utils

import pandas as pd
# import geopandas as gpd
# from shapely.geometry import LineString
# import contextily as ctx
import matplotlib.pyplot as plt

from os.path import dirname, abspath
import os
import sys

In [2]:
torch.manual_seed(1234)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Heroku only have cpu


num_timesteps_input = 8
num_timesteps_output = 4

loss_criterion = nn.MSELoss()

In [3]:
raw_dir = os.path.join(os.getcwd(), 'data', 'raw')
process_dir = os.path.join(os.getcwd(), 'data', 'processed')

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
preprocessing_utils.processed(raw_dir, process_dir, overwrite=True)
A, X, metadata, cat2index, timesteps, means, stds = preprocessing_utils.load(process_dir)


test_original_data = X


test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device).float()

Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_00_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_05_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_10_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_15_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_20_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_predictio

In [4]:
X.shape

(53, 3, 8)

In [5]:
i = 0
indices = [(i, i + (num_timesteps_input + num_timesteps_output))]

# Save samples
features, target = [], []
for i, j in indices:
    features.append(
        X[:, :, i: i + num_timesteps_input].transpose(
            (0, 2, 1)))
    target.append(X[:, 0, i + num_timesteps_input: j])

In [6]:
test_input = torch.from_numpy(np.array(features)).to(device)

In [7]:
test_target = torch.from_numpy(np.array(target)).to(device)

In [8]:
# Load model
# saved_models_path = os.path.join(dirname(os.getcwd()), 'saved_models', 'last_saved_model.txt')
# with open(saved_models_path) as f:
#     saved_model = f.read()

latest_model_path = os.path.join(dirname(os.getcwd()),'saved_models', 'Final_STGCN_Weights')
checkpoint = torch.load(latest_model_path, map_location=None)
model_stgcn = model.Stgcn_Model(checkpoint['model_nodes_num'], checkpoint['model_features_num'], checkpoint['model_input_timesteps'], checkpoint['model_num_output'])
model_stgcn.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model_stgcn.parameters(), lr=checkpoint['model_lr'])
optimizer = optimizer.load_state_dict(checkpoint['opti_state_dict'])

In [9]:
loaded_model = model_stgcn
loaded_model.to(device)
loaded_optimizer = optimizer

In [10]:
test_input.shape

torch.Size([1, 53, 8, 3])

In [11]:
test_target.shape

torch.Size([1, 53, 0])

In [12]:
adj_mat.shape

torch.Size([53, 53])

In [13]:
torch.manual_seed(1234)


predicted = model_utils.predict(loaded_model, test_input, adj_mat)
predicted = predicted.cpu().numpy()
predicted_denorm = preprocessing_utils.denormalize(predicted, stds[0], means[0])

In [14]:
predicted

array([[[-1.415075  , -1.3122897 , -1.1868554 , -1.1799077 ],
        [-0.2656938 , -0.18420967, -0.15164311, -0.06208366],
        [ 0.3780233 ,  0.35590044,  0.29939532,  0.31147975],
        [ 0.36727753,  0.31608406,  0.27773803,  0.26341072],
        [ 0.5446751 ,  0.64699197,  0.646888  ,  0.7025989 ],
        [-0.34256107, -0.5160178 , -0.56228155, -0.57956517],
        [ 0.17994773,  0.29843828,  0.33949316,  0.40767056],
        [-0.6674091 , -0.6707887 , -0.6736678 , -0.6706457 ],
        [ 0.68033016,  0.7212311 ,  0.6754341 ,  0.6736188 ],
        [-0.4586085 , -0.42033264, -0.39433867, -0.39416832],
        [-0.1883955 , -0.10641562, -0.07281922,  0.00360686],
        [-0.81124556, -0.95524955, -1.0005645 , -0.98803055],
        [ 0.0613069 ,  0.21877429,  0.21968885,  0.1617497 ],
        [ 1.3295244 ,  1.0615219 ,  0.8397263 ,  0.71383214],
        [ 0.03048879, -0.13767847, -0.23911159, -0.43037933],
        [ 1.1044369 ,  0.79225373,  0.55076295,  0.33646786],
        

In [15]:
predicted_denorm

array([[[2.4759839, 2.5496788, 2.639613 , 2.6445944],
        [3.3000686, 3.3584912, 3.3818407, 3.4460533],
        [3.7616017, 3.74574  , 3.705227 , 3.713891 ],
        [3.7538972, 3.7171924, 3.689699 , 3.6794267],
        [3.8810878, 3.954447 , 3.9543726, 3.9943163],
        [3.2449563, 3.120591 , 3.0874207, 3.0750287],
        [3.619585 , 3.7045407, 3.7339761, 3.7828581],
        [3.012046 , 3.009623 , 3.0075588, 3.0097256],
        [3.97835  , 4.007675 , 3.9748397, 3.973538 ],
        [3.1617522, 3.1891954, 3.2078326, 3.2079546],
        [3.35549  , 3.414268 , 3.438356 , 3.4931521],
        [2.9089181, 2.80567  , 2.77318  , 2.7821667],
        [3.5345218, 3.647423 , 3.648079 , 3.6065376],
        [4.44381  , 4.2516575, 4.092634 , 4.0023704],
        [3.512426 , 3.391853 , 3.3191276, 3.181992 ],
        [4.2824264, 4.058597 , 3.8854527, 3.7318072],
        [3.2839887, 3.2989728, 3.261142 , 3.221191 ],
        [1.9678814, 2.0516915, 2.172803 , 2.2197516],
        [3.2659266, 3.356622

In [16]:
predicted_denorm

array([[[2.4759839, 2.5496788, 2.639613 , 2.6445944],
        [3.3000686, 3.3584912, 3.3818407, 3.4460533],
        [3.7616017, 3.74574  , 3.705227 , 3.713891 ],
        [3.7538972, 3.7171924, 3.689699 , 3.6794267],
        [3.8810878, 3.954447 , 3.9543726, 3.9943163],
        [3.2449563, 3.120591 , 3.0874207, 3.0750287],
        [3.619585 , 3.7045407, 3.7339761, 3.7828581],
        [3.012046 , 3.009623 , 3.0075588, 3.0097256],
        [3.97835  , 4.007675 , 3.9748397, 3.973538 ],
        [3.1617522, 3.1891954, 3.2078326, 3.2079546],
        [3.35549  , 3.414268 , 3.438356 , 3.4931521],
        [2.9089181, 2.80567  , 2.77318  , 2.7821667],
        [3.5345218, 3.647423 , 3.648079 , 3.6065376],
        [4.44381  , 4.2516575, 4.092634 , 4.0023704],
        [3.512426 , 3.391853 , 3.3191276, 3.181992 ],
        [4.2824264, 4.058597 , 3.8854527, 3.7318072],
        [3.2839887, 3.2989728, 3.261142 , 3.221191 ],
        [1.9678814, 2.0516915, 2.172803 , 2.2197516],
        [3.2659266, 3.356622

In [17]:
predicted_denorm[:,:, :2]

array([[[2.4759839, 2.5496788],
        [3.3000686, 3.3584912],
        [3.7616017, 3.74574  ],
        [3.7538972, 3.7171924],
        [3.8810878, 3.954447 ],
        [3.2449563, 3.120591 ],
        [3.619585 , 3.7045407],
        [3.012046 , 3.009623 ],
        [3.97835  , 4.007675 ],
        [3.1617522, 3.1891954],
        [3.35549  , 3.414268 ],
        [2.9089181, 2.80567  ],
        [3.5345218, 3.647423 ],
        [4.44381  , 4.2516575],
        [3.512426 , 3.391853 ],
        [4.2824264, 4.058597 ],
        [3.2839887, 3.2989728],
        [1.9678814, 2.0516915],
        [3.2659266, 3.3566222],
        [3.47406  , 3.647519 ],
        [3.154872 , 3.1558673],
        [3.7041683, 3.6369438],
        [3.310739 , 3.2739463],
        [3.4475422, 3.3583336],
        [3.8089776, 3.8233023],
        [3.3366327, 3.4035118],
        [3.94236  , 3.8906922],
        [3.5666134, 3.516386 ],
        [3.5350113, 3.4973216],
        [3.4943702, 3.481276 ],
        [3.7796555, 3.7946856],
        

In [18]:
predicted_denorm.shape

(1, 53, 4)

In [19]:
timestep_speedbands = predicted_denorm.reshape(predicted_denorm.shape[2], predicted_denorm.shape[1])

In [20]:
timestep_speedbands

array([[2.4759839, 2.5496788, 2.639613 , 2.6445944, 3.3000686, 3.3584912,
        3.3818407, 3.4460533, 3.7616017, 3.74574  , 3.705227 , 3.713891 ,
        3.7538972, 3.7171924, 3.689699 , 3.6794267, 3.8810878, 3.954447 ,
        3.9543726, 3.9943163, 3.2449563, 3.120591 , 3.0874207, 3.0750287,
        3.619585 , 3.7045407, 3.7339761, 3.7828581, 3.012046 , 3.009623 ,
        3.0075588, 3.0097256, 3.97835  , 4.007675 , 3.9748397, 3.973538 ,
        3.1617522, 3.1891954, 3.2078326, 3.2079546, 3.35549  , 3.414268 ,
        3.438356 , 3.4931521, 2.9089181, 2.80567  , 2.77318  , 2.7821667,
        3.5345218, 3.647423 , 3.648079 , 3.6065376, 4.44381  ],
       [4.2516575, 4.092634 , 4.0023704, 3.512426 , 3.391853 , 3.3191276,
        3.181992 , 4.2824264, 4.058597 , 3.8854527, 3.7318072, 3.2839887,
        3.2989728, 3.261142 , 3.221191 , 1.9678814, 2.0516915, 2.172803 ,
        2.2197516, 3.2659266, 3.3566222, 3.3869858, 3.3956788, 3.47406  ,
        3.647519 , 3.7289026, 3.7426546, 3.15487

In [21]:
def loc_to_linestring(loc):
    coordArr = loc.split()
    coordArr = [float(coord) for coord in coordArr]
    return LineString([coordArr[1::-1], coordArr[3:1:-1]])

def plotGeoPerformance(metadata, speedbands):
    df = pd.DataFrame(metadata).transpose()
    df["speedbands"] = speedbands
    loc = df["start_pos"] + " " + df["end_pos"]
    linestrings = loc.apply(loc_to_linestring)
    gdf = gpd.GeoDataFrame(df, geometry=linestrings, crs="EPSG:4326")
    gdf = gdf.to_crs('EPSG:3857')
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, column="speedbands", legend=True, cmap="OrRd", legend_kwds={'label': 'speedbands'})
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ctx.add_basemap(ax)

In [ ]:
plotGeoPerformance(metadata, timestep_speedbands[0])